In [18]:
import sys
sys.path.append('..')
import importlib
import datagit.drift_evaluators_dry_run
importlib.reload(datagit.drift_evaluators_dry_run)
from datagit.drift_evaluators_dry_run import run_drift_evaluator
from datagit.drift_evaluators import default_drift_evaluator
from dotenv import load_dotenv
from github import Github
import os

load_dotenv("../.env")
gh_token = os.getenv("GH_TOKEN")
gh_client = Github(gh_token)
repo = os.getenv("REPO") or "gh_org/repo"

drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha="b652e7a591cdf621eb9d13933a758293301316eb", drift_evaluator=default_drift_evaluator)
drift

{'should_alert': True,
 'message': 'Drift detected:\n- ~~🆕 0 addition~~\n- ♻️ 6 modifications\n- ~~🗑️ 0 deletion~~'}

In [47]:
import datagit.dataset_helpers
importlib.reload(datagit.dataset_helpers)
from datagit.dataset_helpers import get_addition_deletion_and_diff
import pandas as pd

def example_drift_evaluator(data_drift_context):
    new_dataframe =  data_drift_context["computed_dataframe"]
    old_dataframe =  data_drift_context["reported_dataframe"]
    additions, deletions, diff= get_addition_deletion_and_diff(old_dataframe, new_dataframe, "unique_key")

    print("additions", additions)
    print("deletions", deletions)
    print("diff", diff)
    diff_column = diff.columns[0][0]
    diff = diff.apply(pd.to_numeric)  # Convert the values in the diff dataframe to numeric values
    
    print("computing the diff")
    monthly_booking_drift = diff[(diff_column, "other")] - diff[(diff_column, "self")]
    print("monthly_booking_drift", monthly_booking_drift)
    if(monthly_booking_drift > 1000).any():
        return {"should_alert": True, "message": "The monthly booking drift is too high"}
    return {"should_alert": False, "message": "Small drift"}

# no_drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha="b652e7a591cdf621eb9d13933a758293301316eb", drift_evaluator=example_drift_evaluator)
# print("no_drift", no_drift)

large_drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha="3e04a216677804ec4789db430dfc51b80a31c085", drift_evaluator=example_drift_evaluator)
print("large_drift", large_drift)

additions 0
deletions 0
diff        metric_value      
               self other
84404          5.27  0.39
88429          1.48  6.84
142971         0.09  3.54
181368         0.76  4.32
234161         7.43   6.0
computing the diff
monthly_booking_drift 84404    -4.88
88429     5.36
142971    3.45
181368    3.56
234161   -1.43
dtype: float64
large_drift {'should_alert': False, 'message': 'Small drift'}
